In [ ]:
import numpy  as np
import pandas as pd                     
from   math   import radians as DegToRad       # Degrees to radians Conversion

from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")

from shapely.geometry import Point             # Imported for constraint checking
from shapely.geometry.polygon import Polygon

In [ ]:
def check_distance_conflict(coords, coords_i):
    distance_mat = np.sqrt(np.sum(np.power((coords - coords_i), 2), axis=1))
    if (np.sum(distance_mat <= 400) > 1):
        return False
    else:
        return True
        

def randomly_create_windfarm_grid():
    N_TURBINES = 50

    def get_coordinates(random_shape):
        shape = np.array([random_shape, random_shape])

        width_ratio = shape[1] / shape[0]
        num_y = np.int32(np.sqrt(N_TURBINES / width_ratio)) + 1
        num_x = np.int32(N_TURBINES / num_y) + 1

        x = np.linspace(0., shape[1]-1, num_x, dtype=np.float32)
        y = np.linspace(0., shape[0]-1, num_y, dtype=np.float32)
        coords = np.stack(np.meshgrid(x, y), -1).reshape(-1,2)

        init_dist = np.min((x[1]-x[0], y[1]-y[0]))
        MIN_DIST = 400

        max_movement = (init_dist - MIN_DIST)/2
        noise = np.random.uniform(low=-max_movement, high=max_movement, size=(len(coords), 2))
        coords += noise

        coords[:, 0] = coords[:, 0] + np.abs(np.min(coords[:, 0]))
        coords[:, 1] = coords[:, 1] + np.abs(np.min(coords[:, 1]))
        coords2 = coords[np.random.choice(list(range(0, coords.shape[0])), size=N_TURBINES, replace=False)]
        return coords2

    while(True):
        random_shape = np.random.choice([3600, 3650, 3700], size=1)
        coords2 = get_coordinates(random_shape[0])
        if ((np.sum([check_distance_conflict(coords2, coords2[i]) for i in range(coords2.shape[0])]) == N_TURBINES) & (np.max(coords2) <= 3900) & (np.min(coords2) >= 0)):
            return coords2 + 50           

In [ ]:
def generate_populations(population_size):
    POPs_MATRIX = np.zeros((population_size, 50, 2))
    for i in range(population_size):
        POPs_MATRIX[i] = randomly_create_windfarm_grid()
    return POPs_MATRIX

In [ ]:
def getTurbLoc(turb_loc_file_name):
    df = pd.read_csv(turb_loc_file_name, sep=',', dtype = np.float32)
    turb_coords = df.to_numpy(dtype = np.float32)
    return(turb_coords)

def loadPowerCurve(power_curve_file_name):
    powerCurve = pd.read_csv(power_curve_file_name, sep=',', dtype = np.float32)
    powerCurve = powerCurve.to_numpy(dtype = np.float32)
    return(powerCurve)

def binWindResourceData(wind_data_folder_name):
    final = pd.DataFrame()
    for i in [2007, 2008, 2009, 2013, 2014, 2015, 2017]:
        path = os.path.join(wind_data_folder_name, wind_data_"+str(i)+".csv")
        temp = pd.read_csv(path)
        final = pd.concat([final, temp], axis=0)
    # Load wind data. Then, extracts the 'drct', 'sped' columns
    wind_resource = final[['drct', 'sped']].to_numpy(dtype = np.float32)
    
    # direction 'slices' in degrees
    slices_drct   = np.roll(np.arange(10, 361, 10, dtype=np.float32), 1)
    n_slices_drct = slices_drct.shape[0]
    
    # speed 'slices'
    slices_sped   = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0]
    n_slices_sped = len(slices_sped)-1

    # placeholder for binned wind
    binned_wind = np.zeros((n_slices_drct, n_slices_sped), dtype = np.float32)
    
    # 'trap' data points inside the bins. 
    for i in range(n_slices_drct):
        for j in range(n_slices_sped):     
            foo = wind_resource[(wind_resource[:,0] == slices_drct[i])] 
            foo = foo[(foo[:,1] >= slices_sped[j]) & (foo[:,1] <  slices_sped[j+1])]
            binned_wind[i,j] = foo.shape[0]
    
    wind_inst_freq   = binned_wind/np.sum(binned_wind)
    wind_inst_freq   = wind_inst_freq.ravel()
    return(wind_inst_freq)

                            
def searchSorted(lookup, sample_array):
    lookup_middles = lookup[1:] - np.diff(lookup.astype('f'))/2
    idx1 = np.searchsorted(lookup_middles, sample_array)
    indices = np.arange(lookup.shape[0])[idx1]
    return indices

                            
def preProcessing(power_curve):
    # number of turbines
    n_turbs = 50
    
    # direction 'slices' in degrees
    slices_drct   = np.roll(np.arange(10, 361, 10, dtype=np.float32), 1)
    ## slices_drct   = [360, 10.0, 20.0.......340, 350]
    n_slices_drct = slices_drct.shape[0]
    
    # speed 'slices'
    slices_sped   = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0]
    n_slices_sped = len(slices_sped)-1
    
    # number of wind instances
    n_wind_instances = (n_slices_drct)*(n_slices_sped)
    
    # Create wind instances. There are two columns in the wind instance array
    # First Column - Wind Speed. Second Column - Wind Direction
    # Shape of wind_instances (n_wind_instances,2). 
    # Values [1.,360.],[3.,360.],[5.,360.]...[25.,350.],[27.,350.],29.,350.]
    wind_instances = np.zeros((n_wind_instances,2), dtype=np.float32)
    counter = 0
    for i in range(n_slices_drct):
        for j in range(n_slices_sped): 
            wind_drct =  slices_drct[i]
            wind_sped = (slices_sped[j] + slices_sped[j+1])/2
            wind_instances[counter,0] = wind_sped
            wind_instances[counter,1] = wind_drct
            counter += 1

    # So that the wind flow direction aligns with the +ve x-axis.
    # Convert inflow wind direction from degrees to radians
    wind_drcts =  np.radians(wind_instances[:,1] - 90)
    # For coordinate transformation 
    cos_dir = np.cos(wind_drcts).reshape(n_wind_instances,1)
    sin_dir = np.sin(wind_drcts).reshape(n_wind_instances,1)
    
    # create copies of n_wind_instances wind speeds from wind_instances
    wind_sped_stacked = np.column_stack([wind_instances[:,0]]*n_turbs)
   
    # Pre-prepare matrix with stored thrust coeffecient C_t values for n_wind_instances shape (n_wind_instances, n_turbs, n_turbs). 
    # Value changing only along axis=0. C_t, thrust coeff. values for all speed instances. 
    # we use power_curve data as look up to estimate the thrust coeff. of the turbine for the corresponding closest matching wind speed
    indices = searchSorted(power_curve[:,0], wind_instances[:,0])
    C_t     = power_curve[indices,1]
    # stacking and reshaping to assist vectorization
    C_t     = np.column_stack([C_t]*(n_turbs*n_turbs))
    C_t     = C_t.reshape(n_wind_instances, n_turbs, n_turbs)
    return(n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)

In [ ]:
def getAEP(turb_rad, turb_coords, power_curve, wind_inst_freq, 
            n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t):
    
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Calculates AEP of the wind farm. Vectorised version.
    
    :called from
        main
        
    :param
        turb_diam         - Radius of the turbine (m)
        turb_coords       - 2D array turbine euclidean x,y coordinates
        power_curve       - For estimating power. 
        wind_inst_freq    - 1-D flattened with rough probabilities of 
                            wind instance occurence.
                            n_wind_instances  - number of wind instances (int)
        cos_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        sin_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        wind_sped_stacked - column staked all speed instances n_turb times. 
        C_t               - 3D array with shape (n_wind_instances, n_turbs, n_turbs)
                            Value changing only along axis=0. C_t, thrust coeff.
                            values for all speed instances. 
    
    :return
        wind farm AEP in Gigawatt Hours, GWh (float)
    """
    # number of turbines
    n_turbs        =   turb_coords.shape[0]
    assert n_turbs ==  50, "Error! Number of turbines is not 50."
    
    # Prepare the rotated coordinates wrt the wind direction i.e downwind(x) & crosswind(y) 
    # coordinates wrt to the wind direction for each direction in wind_instances array
    rotate_coords   =  np.zeros((n_wind_instances, n_turbs, 2), dtype=np.float32)
    # Coordinate Transformation. Rotate coordinates to downwind, crosswind coordinates
    rotate_coords[:,:,0] =  np.matmul(cos_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) - \
                           np.matmul(sin_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
    rotate_coords[:,:,1] =  np.matmul(sin_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) +\
                           np.matmul(cos_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
    
    
    # x_dist - x dist between turbine pairs wrt downwind/crosswind coordinates)
    # for each wind instance
    x_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
    for i in range(n_wind_instances):
        tmp = rotate_coords[i,:,0].repeat(n_turbs).reshape(n_turbs, n_turbs)
        x_dist[i] = tmp - tmp.transpose()
    
    
    # y_dist - y dist between turbine pairs wrt downwind/crosswind coordinates)
    # for each wind instance    
    y_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
    for i in range(n_wind_instances):
        tmp = rotate_coords[i,:,1].repeat(n_turbs).reshape(n_turbs, n_turbs)
        y_dist[i] = tmp - tmp.transpose()
    y_dist = np.abs(y_dist) 
     
    
    # Now use element wise operations to calculate speed deficit.
    # kw, wake decay constant presetted to 0.05
    # use the jensen's model formula. 
    # no wake effect of turbine on itself. either j not an upstream or wake 
    # not happening on i because its outside of the wake region of j
    # For some values of x_dist here RuntimeWarning: divide by zero may occur
    # That occurs for negative x_dist. Those we anyway mark as zeros. 
    sped_deficit = (1-np.sqrt(1-C_t))*((turb_rad/(turb_rad + 0.05*x_dist))**2) 
    sped_deficit[((x_dist <= 0) | ((x_dist > 0) & (y_dist > (turb_rad + 0.05*x_dist))))] = 0.0
    
    
    # Calculate Total speed deficit from all upstream turbs, using sqrt of sum of sqrs
    sped_deficit_eff  = np.sqrt(np.sum(np.square(sped_deficit), axis = 2))
    
    
    # Element wise multiply the above with (1- sped_deficit_eff) to get
    # effective windspeed due to the happening wake
    wind_sped_eff     = wind_sped_stacked*(1.0-sped_deficit_eff)

    
    # Estimate power from power_curve look up for wind_sped_eff
    indices = searchSorted(power_curve[:,0], wind_sped_eff.ravel())
    power   = power_curve[indices,2]
    power   = power.reshape(n_wind_instances,n_turbs)
    
    # Farm power for single wind instance 
    power   = np.sum(power, axis=1)
    
    # multiply the respective values with the wind instance probabilities 
    # year_hours = 8760.0
    AEP = 8760.0*np.sum(power*wind_inst_freq)
    
    # Convert MWh to GWh
    AEP = AEP/1e3
    
    return(AEP)

In [ ]:
power_curve = loadPowerCurve("data\power_curve.csv")
wind_inst_freq = binWindResourceData(os.path.join(os.getcwd(),"data"), "Wind Data")
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)


turb_rad = 50

def select_N_best_layouts(POPs_MATRIX, top_n):
    AEP_array = []
    for counter, i in enumerate(range(POPs_MATRIX.shape[0])):
        turb_coords = POPs_MATRIX[i]
        AEP_array.append(getAEP(turb_rad, turb_coords, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t))
    AEP_array = np.array(AEP_array) 
    AEP_array_argsort = np.argsort(AEP_array*-1) #for descending order
    return POPs_MATRIX[AEP_array_argsort[:top_n]], AEP_array[AEP_array_argsort]

In [ ]:
def point_mutation(layout):
    max_iter = 0
    while(max_iter <= 10):
        index = np.random.randint(low=0, high=50, size=1)[0]
        layout_mutated = layout.copy()
        layout_mutated[index] = layout_mutated[index] + np.random.uniform(low=-10, high=10, size=2)
        if ((np.sum([check_distance_conflict(layout_mutated, layout_mutated[i]) for i in range(layout_mutated.shape[0])]) == 50) & (np.max(layout_mutated) <= 3950) & (np.min(layout_mutated) >= 50)):
            return layout_mutated
        max_iter+=1
    return layout

In [ ]:
def cross_mutation(layout_male, layout_female):
    max_iter = 0
    while(max_iter <= 10):
        layout_male_mutated, layout_female_mutated = layout_male.copy(), layout_female.copy()
        
        index_m, index_f = -1, -1
        while(index_m == index_f):
            index_m = np.random.randint(low=0, high=50, size=1)[0]
            index_f = np.random.randint(low=0, high=50, size=1)[0]
        
        x = layout_male_mutated[index_m]
        y = layout_female_mutated[index_f]
        
        layout_male_mutated[index_m][0] = y[0]
        layout_female_mutated[index_f][1] = x[1]
        
        condition1 = (np.sum([check_distance_conflict(layout_male_mutated, layout_male_mutated[i]) for i in range(layout_male_mutated.shape[0])]) == 50) & (np.max(layout_male_mutated) <= 3950) & (np.min(layout_male_mutated) >= 50)
        condition2 = (np.sum([check_distance_conflict(layout_female_mutated, layout_female_mutated[i]) for i in range(layout_female_mutated.shape[0])]) == 50) & (np.max(layout_female_mutated) <= 3950) & (np.min(layout_female_mutated) >= 50)
        
        if ((condition1) & (condition2)):
            return layout_male_mutated, layout_female_mutated
        max_iter += 1
    
    return layout_male, layout_female

In [ ]:
def random_mutate_for_layout(layout):
    max_iter = 0
    while(max_iter <= 5):
        noise = np.random.uniform(low=-3, high=3, size=layout.shape)
        layout_mutated = layout + noise
        condition = (np.sum([check_distance_conflict(layout_mutated, layout_mutated[i]) for i in range(layout_mutated.shape[0])]) == 50) & (np.max(layout_mutated) <= 3950) & (np.min(layout_mutated) >= 50)
        if (condition):
            layout = layout_mutated.copy()
        max_iter += 1
    return layout

In [ ]:
def randomly_alter_the_layout(layout):
    max_iter = 0
    layout_mutated = layout.copy()
    while(max_iter <= 10):
        index_for_random_altration = np.random.randint(low=0, high=50, size=1)[0]
        layout_mutated[index_for_random_altration] = np.random.uniform(low=50, high=3950, size=(1, 2))
        condition = (np.sum([check_distance_conflict(layout_mutated, layout_mutated[i]) for i in range(layout_mutated.shape[0])]) == 50) & (np.max(layout_mutated) <= 3950) & (np.min(layout_mutated) >= 50)
        if (condition):
            layout = layout_mutated.copy()
        max_iter += 1
    return layout

In [ ]:
current_timestamp = str(str(datetime.now()).split(".")[0].replace(":", "_"))
op_folder_path = os.path.join(os.getcwd(), current_timestamp)
if not os.path.exists(op_folder_path):
    os.makedirs(op_folder_path)

In [ ]:
POPULATION_SIZE = 200
elite_rate = 0.1
layout_mutate = 0.4
point_mutate = 0.5
crossover_mutate = 0.5
random_rate = 0.3
num_of_generations = 1000


print ("Started GA...")

POPs_MATRIX = generate_populations(2*POPULATION_SIZE)

print ("Selecting Initial Populations...")
best_POPs_MATRIX, _ = select_N_best_layouts(POPs_MATRIX, POPULATION_SIZE)



best_AEP = 0
AEP_unchanged_counter = 0

for generation in range(0, num_of_generations):
    print ("Generation - "+str(generation))
    l = np.arange(0, best_POPs_MATRIX.shape[0])
    np.random.shuffle(l)

    elite_layout_number = int(POPULATION_SIZE*elite_rate)
    layout_mutate_number = int(POPULATION_SIZE*layout_mutate)
    point_mutate_number = int(POPULATION_SIZE*point_mutate)
    cross_mutation_number = int(POPULATION_SIZE*crossover_mutate)
    random_population_number = int(POPULATION_SIZE*random_rate)
    
    indexes_for_cross_mutation = np.arange(cross_mutation_number)
    
    
    #elite candidates
    best_POPs_MATRIX_elite = best_POPs_MATRIX[:elite_layout_number].copy()
    
    
    #layout mutation
    best_POPs_MATRIX_layout_mutate = best_POPs_MATRIX[:layout_mutate_number].copy()
    for i in range(best_POPs_MATRIX_layout_mutate.shape[0]):
        best_POPs_MATRIX_layout_mutate[i] = random_mutate_for_layout(best_POPs_MATRIX_layout_mutate[i])
    
    
    #point mutation
    best_POPs_MATRIX_for_point_mutation = best_POPs_MATRIX[:point_mutate_number].copy()
    for i in range(best_POPs_MATRIX_for_point_mutation.shape[0]):
        best_POPs_MATRIX_for_point_mutation[i] = point_mutation(best_POPs_MATRIX[i])
    
    
    #cross-over mutation
    best_POPs_MATRIX_for_crossover_mutation = best_POPs_MATRIX[:cross_mutation_number].copy()
    for i in range(best_POPs_MATRIX_for_crossover_mutation.shape[0]):
        cross_mutation_pair = np.random.choice(indexes_for_cross_mutation, size=(2), replace=False)
        best_POPs_MATRIX_for_crossover_mutation[cross_mutation_pair[0]], best_POPs_MATRIX_for_crossover_mutation[cross_mutation_pair[1]] = cross_mutation(best_POPs_MATRIX_for_crossover_mutation[cross_mutation_pair[0]], best_POPs_MATRIX_for_crossover_mutation[cross_mutation_pair[1]])
    
    
    #random mutation
    best_POPs_MATRIX_for_random_mutation = best_POPs_MATRIX[:random_population_number].copy()
    for i in range(best_POPs_MATRIX_for_random_mutation.shape[0]):
        best_POPs_MATRIX_for_random_mutation[i] = randomly_alter_the_layout(best_POPs_MATRIX_for_random_mutation[i])
    
    
    POPs_MATRIX_generationwise = np.vstack([best_POPs_MATRIX_elite, best_POPs_MATRIX_layout_mutate, best_POPs_MATRIX_for_point_mutation, best_POPs_MATRIX_for_crossover_mutation, best_POPs_MATRIX_for_random_mutation])
    best_POPs_MATRIX, AEP_array = select_N_best_layouts(POPs_MATRIX_generationwise, POPULATION_SIZE)
    
    
    print ("Best AEP in generation "+str(generation)+" is : "+str(AEP_array[0]))
    if (best_AEP < AEP_array[0]):
        best_AEP = AEP_array[0]
        AEP_unchanged_counter = 0
    else:
        AEP_unchanged_counter += 1
    
    if (AEP_unchanged_counter == 200):
        print ("AEP did'nt change for 200 iterations... Stopping!")
        break
    
    #save the layouts after every 50 generations
    if (generation%50 == 0):
        np.save(os.path.join(op_folder_path, "generation_"+str(generation)+".npy"), best_POPs_MATRIX)